In [ ]:
! pip install chatterbot
! pip install chatterbot_corpus

     |████████████████████████████████| 71kB 5.4MB/s 
     |████████████████████████████████| 1.3MB 22.8MB/s 
  Found existing installation: SQLAlchemy 1.4.18
    Uninstalling SQLAlchemy-1.4.18:
      Successfully uninstalled SQLAlchemy-1.4.18
     |████████████████████████████████| 122kB 14.3MB/s 


In [ ]:
pip install newspaper3k

     |████████████████████████████████| 215kB 13.3MB/s 
     |████████████████████████████████| 7.4MB 14.0MB/s 
     |████████████████████████████████| 92kB 8.8MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398404 sha256=f7cc9a83c1ab414ff1ddb04ef89c17530eed55e8540ed64e644b8f5d0c2a0fbf
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=680b57df07cb750a7fc08a2e324a91749b7d5c2630ff622f51de4bf3942edd28
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13553 sha256=0a81a39ade965665cf8bc0faf621e58b55965fc41f0322048e141b2b5feef126
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7

In [ ]:
pip install --user -U nltk

Requirement already up-to-date: nltk in /root/.local/lib/python3.7/site-packages (3.6.2)


Import Libraries

In [ ]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
# not used

import nltk
nltk.download('punkt')
nltk.download('wordnet')
import numpy as np
import random
import string
import bs4 as bs
import requests, json
import urllib.request
import re
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import warnings
warnings.filterwarnings("ignore")

import click

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Specifying the target for additional knowledge

In [ ]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Weather')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [ ]:
article_text

'\nweather is the state of the atmosphere, describing for example the degree to which it is hot or cold, wet or dry, calm or stormy, clear or cloudy.[1] on earth, most weather phenomena occur in the lowest layer of the planet\'s atmosphere, the troposphere,[2][3] just below the stratosphere. weather refers to day-to-day temperature, precipitation, and other atmospheric conditions, whereas climate is the term for the averaging of atmospheric conditions over longer periods of time.[4] when used without qualification, "weather" is generally understood to mean the weather of earth.\nweather is driven by air pressure, temperature, and moisture differences between one place and another. these differences can occur due to the sun\'s angle at any particular spot, which varies with latitude. the strong temperature contrast between polar and tropical air gives rise to the largest scale atmospheric circulations: the hadley cell, the ferrel cell, the polar cell, and the jet stream.  weather system

In [ ]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)

In [ ]:
#Throws an error and I am not sure why

article_sentences = nltk.sent_tokenize(article_text)
article_words = nltk.word_tokenize(article_text)

In [ ]:
wnlemmatizer = nltk.stem.WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [ ]:
greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup","hello","yo","what it do","what it is","wa up","sup")
greeting_responses = ["hey", "hey hows you?", "*nods* Sup?", "hello, how you doing", "hello", "Welcome, I am good and you"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [ ]:
def generate_response(user_input):
    weatherrobo_response = ''
    article_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(article_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        weatherrobo_response = weatherrobo_response + "I am sorry, but what does that have to do with what we are talking about?"
        return weatherrobo_response
    else:
        weatherrobo_response = weatherrobo_response + article_sentences[similar_sentence_number]
        return weatherrobo_response

In [ ]:
#should have some custom stop words here
word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
all_word_vectors = word_vectorizer.fit_transform(article_sentences)

In [ ]:
similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
similar_vector_values

array([[0.0281538 , 0.03632804, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.02906955, 0.        , 0.        , 0.03475772, 0.        ,
        0.        , 0.03562151, 0.        , 0.08216305, 0.11263766,
        0.04557665, 0.        , 0.04904988, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03023476, 0.        ,
        0.03625953, 0.05268017, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03267926, 0.07491829,
        0.        , 0.09118946, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.10005657, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.02255035, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03

In [ ]:
similar_sentence_number = similar_vector_values.argsort()[0][-2]
similar_sentence_number

122

Connecting to Weather API to get weather for local information

In [ ]:
BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
CITY = "New York"
API_KEY = "80500bda3ab49ea41fa2c4e3b7afa84a"


URL = BASE_URL + "q=" + CITY +"&units=imperial"+"&appid=" + API_KEY
response = requests.get(URL)
if response.status_code == 200:
   data = response.json()
   main = data['main']
   temperature = main['temp']
   humidity = main['humidity']
   pressure = main['pressure']
   report = data['weather']
   print(f"{CITY:}")
   print(f"Temperature: {temperature}")
   print(f"Humidity: {humidity}")
   print(f"Pressure: {pressure}")
   print(f"Weather Report: {report[0]['description']}")
else:
   # showing the error message
   print("Error in the HTTP request")



New York
Temperature: 82.53
Humidity: 56
Pressure: 1010
Weather Report: clear sky


In [ ]:
tempy=str(round(temperature,0))

In [ ]:
random_question = [' What kind of weather is best for a cookout? ', ' When is too cold to wear shorts? ',
                   ' Have you ever gotten drunk? ',' What is the weather outside like to you?',
                   ' What is your ideal weather like? ']

random_question_selection3=random.sample(random_question,1)

In [ ]:
question=random_question_selection3[0]

Chat Bot itself

In [ ]:
#Hello message

In [ ]:
choices={'Sunny':1,'Rainy':2 }
choice = click.prompt('What would you rather have?', type=click.Choice(choices, case_sensitive=False))
print(f'That is great! I like them both because well I am a machine...I am not sure what I would like if I was a human. I have another question for you!')

What would you rather have? (Sunny, Rainy): Sunny
That is great! I like them both because well I am a machine...I am not sure what I would like if I was a human. I have another question for you!


In [ ]:
print(f"Bot: Have you been outside Today is {report[0]['description']} and it is around {tempy} degrees.")
user_input1=input("What do you think of that?\n")

Bot: Have you been outside Today is clear sky and it is around 83.0 degrees.
What do you think of that?
I think it stinks


In [ ]:
continue_dialogue = True
print("Hello, I am your friend. You can ask me any question regarding weather:")
while(continue_dialogue == True):
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            continue_dialogue = False
            print(f"Bot: You are welcome go out have fun it is {str(temperature)} degrees and it is {str(report[0]['description'])} outside")
        else:
            if generate_greeting_response(human_text) != None:
                print(f"Bot: {generate_greeting_response(human_text)} Today's about {str(temperature)} degrees and it is {str(report[0]['description'])} outside")
            else:
                print("Bot: ", end="")
                print(generate_response(human_text))
                article_sentences.remove(human_text)
    else:
        continue_dialogue = False
        print("Bot: Good bye and take care of yourself...")

Hello, I am your friend. You can ask me any question regarding weather:
yo
Bot: Welcome, I am good and you Today's about 82.53 degrees and it is clear sky outside
wow dude it is really hot outside is that because of global warming?
Bot: the range of climates and latitudes across the planet can offer extremes of temperature outside this range.
so the answer to the earth getting hotter is ?
Bot: weather is one of the fundamental processes that shape the earth.
Okay so you are a bot
Bot: I am sorry, but what does that have to do with what we are talking about?
Thanks
Bot: You are welcome go out have fun it is 82.53 degrees and it is clear sky outside


In [ ]:
#good bye message